# Loop Workflow Example 2

> * High level approach to making a 3D model from just a bounding box you can draw
> * To run this notebook for the first time, there are some dependencies needed to use the interactive map. Execute the cell immediately below and restart jupyter.

In [1]:
#if not already installed:
#!conda install -c loop3d map2loop loopstructural pyamg meshio
#!pip install loopstructuralvisualisation[jupyter]

# note on versions:

## map2loop version = v3.2.0
## LoopStructural version = 1.6.5
## LoopProjectFile = 0.2.2
## loopstructural-visualisation = 0.1.10


#!conda install -c conda-forge leafmap ipyleaflet ipywidgets -y
# and
#!pip install geoh5py


import warnings
warnings.filterwarnings('ignore')

## Leaflet Maps

In [2]:
import ipywidgets as widgets
import os
import matplotlib.pyplot as plt

# load last saved map area and model engine (if they exist)
if(not os.path.isdir('../scratch/')):
    os.mkdir('../scratch/')
if(os.path.isfile('../scratch/last_choices.txt')):
    f=open('../scratch/last_choices.txt','r')
    contents =f.readlines()
    f.close()
    default_map=contents[0].replace("\n","")
    default_engine=contents[1].replace("\n","")
else:
    default_map='Turner_Syncline'
    default_engine='loopstructural'

options=['Draw Your Own','Last Area Drawn']

if(not default_map in options):
    default_map= options[0]

map_choice=widgets.Dropdown(
    options=options,
    value=default_map,
    description='Map area:',
    disabled=False,
)
display(map_choice)

Dropdown(description='Map area:', options=('Draw Your Own', 'Last Area Drawn'), value='Draw Your Own')

In [3]:
test_data_name=map_choice.value
print(test_data_name)

Draw Your Own


In [4]:
import leafmap
import pandas as pd
from shapely.geometry import Polygon
import geopandas as gpd
src_crs = "epsg:4326"  # coordinate reference system for imported dtms (geodetic lat/long WGS84)
dst_crs = "epsg:28350"  # coordinate reference system for imported dtms (geodetic lat/long WGS84)

if(not test_data_name =='Draw Your Own'):
    if(test_data_name=='Last Area Drawn'):
        last_coords=pd.read_csv('../scratch/last_area.csv')
        display(last_coords)
        minx=last_coords.iloc[0]['minx']
        miny=last_coords.iloc[0]['miny']
        maxx=last_coords.iloc[0]['maxx']
        maxy=last_coords.iloc[0]['maxy']
    elif(not test_data_name =='Draw Your Own'):
        y_point_list = [miny, miny, maxy, maxy, maxy]
        x_point_list = [minx, maxx, maxx, minx, minx]
        bbox_geom = Polygon(zip(x_point_list, y_point_list))
        polygon = gpd.GeoDataFrame(index=[0], crs=dst_crs, geometry=[bbox_geom])
        polygon_ll=polygon.to_crs(src_crs)

        minx=polygon_ll.total_bounds[0]
        maxx=polygon_ll.total_bounds[2]
        miny=polygon_ll.total_bounds[1]
        maxy=polygon_ll.total_bounds[3]

        minlong=minx
        maxlong=maxx
        minlat=miny
        maxlat=maxy
        #print("x",polygon_ll.total_bounds[0])
        st_bbox=[minlong,minlat,maxlong,maxlat]
        lat_point_list = [minlat, minlat, maxlat, maxlat,maxlat]
        lon_point_list = [minlong, maxlong, maxlong, minlong, minlong]
        bbox_geom = Polygon(zip(lon_point_list, lat_point_list))
        rect = gpd.GeoDataFrame(index=[0], crs=src_crs, geometry=[bbox_geom]) 

    bbox2=str(minx)+","+str(miny)+","+str(maxx)+","+str(maxy)
    y_point_list = [miny, miny, maxy, maxy, maxy]
    x_point_list = [minx, maxx, maxx, minx, minx]
    bbox_geom = Polygon(zip(x_point_list, y_point_list))
    polygon = gpd.GeoDataFrame(index=[0], crs=dst_crs, geometry=[bbox_geom])
    polygon_ll=polygon.to_crs(src_crs)

    minlong=polygon_ll.total_bounds[0]
    maxlong=polygon_ll.total_bounds[2]
    minlat=polygon_ll.total_bounds[1]
    maxlat=polygon_ll.total_bounds[3]
    
    minlong=minx
    maxlong=maxx
    minlat=miny
    maxlat=maxy

    lat_point_list = [minlat, minlat, maxlat, maxlat,maxlat]
    lon_point_list = [minlong, maxlong, maxlong, minlong, minlong]
    bbox_geom = Polygon(zip(lon_point_list, lat_point_list))
    rect = gpd.GeoDataFrame(index=[0], crs=src_crs, geometry=[bbox_geom]) 

    center=(minlat+((maxlat-minlat)/2),minlong+((maxlong-minlong)/2))
else:
    center=(-22.6,117.3)

m = leafmap.Map( center=center, zoom=8,scroll_wheel_zoom=True)
m.add_basemap( basemap='OpenTopoMap')
m.add_wms_layer(url='https://www.loopwms.xyz/geoserver/loop/wms?',
    layers='2_5m_interpgeop15_4326',
    format='image/png',
    transparent=True,
    opacity=0.4,
    attribution='Geology data from GSWA',
    name='geology',
    shown=True)
# url = 'https://www.loopwms.xyz/geoserver/loop/wms?'
m.add_wms_layer(url='https://www.loopwms.xyz/geoserver/loop/wms?',
    layers='2_5m_interpstrucl15_4326',
    format='image/png',
    transparent=True,
    opacity=0.4,
    attribution='Linear features data from GSWA',
    name='faults/folds')
m.add_wms_layer(url='https://www.loopwms.xyz/geoserver/loop/wms?',
    layers='waroxi_wa_4326_bed',
    format='image/png',
    transparent=True,
    attribution='Outcrop data from GSWA',
    name='outcrops')

if( not test_data_name =='Draw Your Own'):
    m.add_gdf(rect, layer_name="Last Rectangle", 
                   fill_colors=['orange'])


In [5]:
polyclip=False # implemented in future version

if(test_data_name=='Draw Your Own' or test_data_name=='Last Area Drawn'):
    if(test_data_name=='Draw Your Own'):
        src_crs = "epsg:4326"  # coordinate reference system for imported dtms (geodetic lat/long WGS84)
        dst_crs = "epsg:28350" # coordinate system for example data
        
        rect=m.draw_features

        if(len(rect)==0):
            minlong=117.13698
            maxlong=117.564464
            minlat= -22.690712
            maxlat=-22.396454

        else:
            minlong=rect[0]['geometry']['coordinates'][0][0][0]
            maxlong=rect[0]['geometry']['coordinates'][0][2][0]
            minlat= rect[0]['geometry']['coordinates'][0][0][1]
            maxlat= rect[0]['geometry']['coordinates'][0][1][1]    
    else:
        use_roi_clip=False
        roi_clip_path=''

    bounds=(minlong,maxlong,minlat,maxlat)

    lat_point_list = [minlat, minlat, maxlat, maxlat,maxlat]
    lon_point_list = [minlong, maxlong, maxlong, minlong, minlong]
    bbox_geom = Polygon(zip(lon_point_list, lat_point_list))
    mbbox = gpd.GeoDataFrame(index=[0], crs=src_crs, geometry=[bbox_geom]) 
    bbox=mbbox.total_bounds
    st_bbox=[bbox[0],bbox[1],bbox[2],bbox[3]]
    print(src_crs,mbbox.total_bounds)
    mbbox=mbbox.to_crs(dst_crs)
    print(dst_crs,mbbox.total_bounds)
    
    f=open('../scratch/last_area.csv','w') 
    ostr='minx,miny,maxx,maxy\n'
    f.write(ostr)
    ostr=str(minlong)+','+str(minlat)+','+str(maxlong)+','+str(maxlat)+'\n'
    f.write(ostr)
    f.close()
    

epsg:4326 [117.13698  -22.690712 117.564464 -22.396454]
epsg:28350 [ 514069.70809875 7490607.48994945  558101.7853635  7523283.65581696]


## Map2Loop

In [10]:
import os
from map2loop.project import Project
from map2loop.m2l_enums import VerboseLevel
from datetime import datetime
from map2loop.thickness_calculator import InterpolatedStructure
import time
import warnings


t0 = time.time()
warnings.filterwarnings('ignore')
nowtime=datetime.now().isoformat(timespec='minutes')
bbox_3d={
    "minx": mbbox.total_bounds[0], #500000,
    "miny": mbbox.total_bounds[1], #7490000,
    "maxx": mbbox.total_bounds[2], #545000,
    "maxy": mbbox.total_bounds[3], #7520000,
    "base": -4800,
    "top": 1200,
}
model_name=os.path.join(".",nowtime.replace(":","-").replace("T","-"))
loop_project_filename=os.path.join("wa_output.loop3d")
proj = Project(
    use_australian_state_data = "WA",
    verbose_level = VerboseLevel.NONE,
    tmp_path  = model_name,
    working_projection = "EPSG:28350",
    bounding_box = bbox_3d,
    loop_project_filename = loop_project_filename, 
    overwrite_loopprojectfile = True
)
proj.set_thickness_calculator(InterpolatedStructure())
proj.run_all(take_best=True)

t1 = time.time()
m2l_time = t1-t0
m2l_string = f"{m2l_time} seconds" if m2l_time < 60 else f"{m2l_time/60.0} minutes"
print(f"map2loop {m2l_string}")

map2loop 58.82671928405762 seconds


## Loop Structural

In [11]:
import LoopProjectFile as LPF
import LoopStructural
from LoopStructural.modelling.input.project_file import LoopProjectfileProcessor as LPFProcessor
from LoopStructural.visualisation import Loop3DView
import numpy as np

# LoopStructural Modelling
fault_params = {
    'interpolatortype':'FDI',
    'nelements':1e4,
}
foliation_params = {
    'interpolatortype':'FDI' , # 'interpolatortype':'PLI',
    'nelements':1e5,  # how many tetras/voxels
    'regularisation':5,
}
projFile = LPF.ProjectFile(loop_project_filename)
processedData = LPFProcessor(projFile)

ERROR: 2024-12-17 16:30:46,834: process_data.py:460 -- Thickness for Pyradie_Formation is less than or equal to 0
 Update the thickness value for Pyradie_Formation before continuing


In [13]:
processedData.thicknesses['Pyradie_Formation']=600

In [14]:
processedData.foliation_properties['sg'] = foliation_params
processedData.fault_properties['interpolatortype'] = fault_params['interpolatortype']
processedData.fault_properties['nelements'] = fault_params['nelements']

model = LoopStructural.GeologicalModel.from_processor(processedData)
model.nsteps=np.array([100,100,50])
model.update()

                    Defaulting to a dip of 90vertical fault
                          projected onto fault surface estimating from fault normal
                    Defaulting to a dip of 90vertical fault
                          projected onto fault surface estimating from fault normal
                    Defaulting to a dip of 90vertical fault
                          projected onto fault surface estimating from fault normal
                    Defaulting to a dip of 90vertical fault
                          projected onto fault surface estimating from fault normal
                    Defaulting to a dip of 90vertical fault
                          projected onto fault surface estimating from fault normal
                    Defaulting to a dip of 90vertical fault
                          projected onto fault surface estimating from fault normal
                    Defaulting to a dip of 90vertical fault
                          projected onto fault surface estimating from fault

  0%|          | 0/31 [00:00<?, ?it/s]

In [15]:
view = Loop3DView(model)
view.plot_model_surfaces()
view.display()

Widget(value='<iframe src="http://localhost:52981/index.html?ui=P_0x1b99d7dd690_0&reconnect=auto" class="pyvis…